In [2]:
#Loading required libraries.
import pandas as pd
import numpy as np
import re
import string
from pprint import pprint
from collections import Counter

#Loading nltk library and its packages for text processing.
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

#Downloading some necessary word lists.
#nltk.download('wordnet')
#nltk.download('stopwords')

#Downloading all dependencies and corpora.
#nltk.download()
#nltk.download('all', halt_on_error=False)

#Importing pattern and its dependencies.
import pattern
from pattern.en import tag
from pattern.en import parse
from pattern.en import pprint


In [3]:
#Need to install this everytime, before using pattern!
pip install setup.py

SyntaxError: invalid syntax (<ipython-input-3-4b5d7af4e521>, line 2)

In [4]:
#Reading Training and Testing Datasets.
train_data = pd.read_csv("C:/Users/Bhair/OneDrive - University of Oklahoma/Master_of_Science_Data/Data Science Practice/Twitter Sentiment Analysis/Data/tweet-sentiment-extraction/train.csv")
test_data = pd.read_csv("C:/Users/Bhair/OneDrive - University of Oklahoma/Master_of_Science_Data/Data Science Practice/Twitter Sentiment Analysis/Data/tweet-sentiment-extraction/test.csv")



#Contraction directory.

CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}




#Function for expanding contractions using above dictonary of contractions.
def expand_contractions(sentence, contraction_mapping):
    #Creating a list of contraction keys.
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags = re.IGNORECASE|re.DOTALL)
    
    #Function for expanding the contractions.
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = (contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower()))
        expanded_contraction = first_char + expanded_contraction[1:]
        return expanded_contraction
    expanded_sentence = contractions_pattern.sub(expand_match, sentence)
    return expanded_sentence



#Function for creating a list ofstrings from the input. It creates different elements in the list if there is a space.
#Basically, a sentence will be converted into a list of words.
def words(text): return re.findall(r'\w+', text.lower())

#Reading the file with vocabulary.
WORDS = Counter(words(open('C:/Users/Bhair/OneDrive - University of Oklahoma/Master_of_Science_Data/Data Science Practice/Twitter Sentiment Analysis/big.txt').read()))

#Function for calculating probability of the given word to find in the vocabulary.
def P(word, N = sum(WORDS.values())): 
    "Probability of `word`."
    #WORDS[word] gives the count of word
    #N is total number of words in the vocabulary.
    return WORDS[word] / N

#Main function which should be called for word correction.
def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key = P)

#Function for genetraing all possible real words from given string.
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

#Function for editing given string. This returns words which are one edit away from input string.
#The function edits1 returns a set of all the edited strings (whether words or not) that can be made with one simple edit: 
#a simple edit to a word is a deletion (remove one letter), a transposition (swap two adjacent letters), a replacement (change one letter to another) or an insertion (add a letter).
#The output of this function can be a big set. For a word of length n, there will be n deletions, n-1 transpositions, 
#26n alterations, and 26(n+1) insertions, for a total of 54n+25 (of which a few are typically duplicates)
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    #Splitting the letters of a word to form a combination.
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

#Function for generating set of words which are two edits away from input string.
def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


#Function for converting a list to a string.
def listToString(inlist):
    #Initialize an empty string.
    string = " "
    return (string.join(inlist))



In [5]:

#Function for preprocessing and normalizing text data.
def data_normalization(train_data):
    #Setting stop word variable using NLTK package.
    stopWords = set(stopwords.words('english'))
    
    #Creating list of punctuations.
    punctuations = [".", ",", "!", "`", "...", "****", ":", "(", ")", "?", "-", "{", "}", "[", "]", ";", "/", "+", "<", 
               ">", "|", "=" "_", "^", "@", "#", "$", "%", "~", "'", ".."]
    
    #Regular expression tokenizer with only alphanumeric contents.
    Token_Pattern = r'\w+'
    regex_wt = nltk.RegexpTokenizer(pattern = Token_Pattern, gaps = False)

    #Listing out all contractions.
    contaction_keys = list(CONTRACTION_MAP.keys())

    #Declaring list for storing lemmatized tweets.
    lemmatized_words = []
    for i in range(len(train_data["text"])):
        #Condition for empty tweets.
        if (str(train_data["text"][i]) == "nan"):
            #Fake strink in the blank tweet.
            train_data["text"][i] = "NA"
        #Expanding contractions.
        #print("Before correction: ", train_data["text"][i])
        text_segment = train_data["text"][i].replace("`", "'")
        expanded_tweets = expand_contractions(text_segment, CONTRACTION_MAP)
        #Removing URL links from tweets.
        expanded_tweets = re.sub(r'http\S+', '', expanded_tweets, flags = re.MULTILINE)
        #print("After contractions: ", expanded_tweets)
        #Tokenizing the words.
        tokens = regex_wt.tokenize(expanded_tweets)
        #Declaring list for lemmatized words.
        lemm_list = []
        for word in tokens:
            #Condition for stopwords.
            if (word not in stopWords):
                #Condition for punctuations.
                if (word not in punctuations):
                    #Correcting the word.
                    #Not using this function because its not correcting words properly. Need to find more accurate way!
                    #corrected_spell = correction(word)
                    corrected_spell = word
                    #Converting all letters to lower form.
                    corrected_spell = corrected_spell.lower()
                    #Lemmatizing words.
                    lemm_list.append(wordnet_lemmatizer.lemmatize(corrected_spell, pos = "v"))
        #Converting the lemmatized word's (document) list to a string.
        lemm_string = listToString(lemm_list)
        lemmatized_words.append(lemm_string)
    return lemmatized_words


In [211]:
CORPUS = data_normalization(train_data[:10])
CORPUS

['i would respond i go',
 'sooo sad i miss san diego',
 'boss bully',
 'interview leave alone',
 'sons could put release already buy',
 'shameless plug best rangers forum earth',
 '2am feed baby fun smile coo',
 'soooo high',
 'both',
 'journey wow u become cooler hehe possible']

In [141]:
test_data["text"][:10]

0    Last session of the day  http://twitpic.com/67ezh
1     Shanghai is also really exciting (precisely -...
2    Recession hit Veronique Branquinho, she has to...
3                                          happy bday!
4               http://twitpic.com/4w75p - I like it!!
5                      that`s great!! weee!! visitors!
6              I THINK EVERYONE HATES ME ON HERE   lol
7     soooooo wish i could, but im in school and my...
8     and within a short time of the last clue all ...
9     What did you get?  My day is alright.. haven`...
Name: text, dtype: object

In [212]:
new_doc = data_normalization(test_data[:10])
new_doc

['last session day',
 'shanghai also really excite precisely skyscrapers galore good tweeps china sh bj',
 'recession hit veronique branquinho quit company shame',
 'happy bday',
 'i like',
 'great weee visitors',
 'i think everyone hat me on here lol',
 'soooooo wish could im school myspace completely block',
 'within short time last clue',
 'what get my day alright do anything yet leave soon stepsister though']

In [197]:
#CORPUS = ['the sky is blue', 'sky is blue and sky is beautiful', 'the beautiful sky is so blue', 'i love blue cheese']
#new_doc = ['loving this blue sky today']

In [213]:
#Bag of words

from sklearn.feature_extraction.text import CountVectorizer

#Function for vectorizing a given document using "Bag of words".
def bow_extractor(corpus, ngram_range = (1, 1)):
    vectorizer = CountVectorizer(min_df=1, ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features


In [214]:
#Building bow vectorizer and extracting features.
bow_vectorizer, bow_features = bow_extractor(CORPUS)

#Densing features.
features = bow_features.todense()
print(features)

#Extracting features from test dataset using above built vectorizer.
new_doc_features = bow_vectorizer.transform(new_doc)
#Densing features.
new_doc_features = new_doc_features.todense()
print(new_doc_features)

#Print the feature names.
feature_names = bow_vectorizer.get_feature_names()
print(feature_names)


[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0
  1 0 0 0]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]
 [0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1
  0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0
  0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
  0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 1]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [215]:
#Function for displaying features.
def display_features(features, feature_names):
    df = pd.DataFrame(data = features, columns = feature_names)
    print(df)

In [216]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
#from feature_extractor import tfidf_transformer
from sklearn.feature_extraction.text import TfidfTransformer

#Function for getting the tfidf-based feature vectors, using Bag of Words feature vectors obtained from above function.
def tfidf_transformer(bow_matrix):
    transformer = TfidfTransformer(norm = 'l2', smooth_idf=True, use_idf=True)
    tfidf_matrix = transformer.fit_transform(bow_matrix)
    return transformer, tfidf_matrix

feature_names = bow_vectorizer.get_feature_names()
#Building tfidf transformer.
tfidf_trans, tdidf_features = tfidf_transformer(bow_features)

features = np.round(tdidf_features.todense(), 2)
display_features(features, feature_names)

#Showing tfidf features for test dataset using built tfidf transformer.
nd_tfidf = tfidf_trans.transform(new_doc_features)
nd_features = np.round(nd_tfidf.todense(), 2)
display_features(nd_features, feature_names)


    2am  alone  already  baby  become  best  boss  both  bully   buy  ...  \
0  0.00   0.00     0.00  0.00    0.00  0.00  0.00   0.0   0.00  0.00  ...   
1  0.00   0.00     0.00  0.00    0.00  0.00  0.00   0.0   0.00  0.00  ...   
2  0.00   0.00     0.00  0.00    0.00  0.00  0.71   0.0   0.71  0.00  ...   
3  0.00   0.58     0.00  0.00    0.00  0.00  0.00   0.0   0.00  0.00  ...   
4  0.00   0.00     0.41  0.00    0.00  0.00  0.00   0.0   0.00  0.41  ...   
5  0.00   0.00     0.00  0.00    0.00  0.41  0.00   0.0   0.00  0.00  ...   
6  0.41   0.00     0.00  0.41    0.00  0.00  0.00   0.0   0.00  0.00  ...   
7  0.00   0.00     0.00  0.00    0.00  0.00  0.00   0.0   0.00  0.00  ...   
8  0.00   0.00     0.00  0.00    0.00  0.00  0.00   1.0   0.00  0.00  ...   
9  0.00   0.00     0.00  0.00    0.41  0.00  0.00   0.0   0.00  0.00  ...   

   respond   sad   san  shameless  smile  sons  sooo  soooo  would   wow  
0     0.58  0.00  0.00       0.00   0.00  0.00  0.00   0.00   0.58  0.00  
1 

In [217]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Building fuction for extracting Tf-Idf based features.
def tfidf_extractor(corpus, ngram_range=(1,1)):
    #Using L2 norm.
    vectorizer = TfidfVectorizer(min_df = 1, norm = 'l2', smooth_idf = True, use_idf = True, ngram_range = ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

In [218]:
#Building tfidf vectorizer and getting training corpus feature vectors.
tfidf_vectorizer, tdidf_features = tfidf_extractor(CORPUS)
features = np.round(tdidf_features.todense(), 2)
display_features(features, feature_names)

#Getting tfidf feature vector for the test dataset.
nd_tfidf = tfidf_vectorizer.transform(new_doc)
display_features(np.round(nd_tfidf.todense(), 2), feature_names)

    2am  alone  already  baby  become  best  boss  both  bully   buy  ...  \
0  0.00   0.00     0.00  0.00    0.00  0.00  0.00   0.0   0.00  0.00  ...   
1  0.00   0.00     0.00  0.00    0.00  0.00  0.00   0.0   0.00  0.00  ...   
2  0.00   0.00     0.00  0.00    0.00  0.00  0.71   0.0   0.71  0.00  ...   
3  0.00   0.58     0.00  0.00    0.00  0.00  0.00   0.0   0.00  0.00  ...   
4  0.00   0.00     0.41  0.00    0.00  0.00  0.00   0.0   0.00  0.41  ...   
5  0.00   0.00     0.00  0.00    0.00  0.41  0.00   0.0   0.00  0.00  ...   
6  0.41   0.00     0.00  0.41    0.00  0.00  0.00   0.0   0.00  0.00  ...   
7  0.00   0.00     0.00  0.00    0.00  0.00  0.00   0.0   0.00  0.00  ...   
8  0.00   0.00     0.00  0.00    0.00  0.00  0.00   1.0   0.00  0.00  ...   
9  0.00   0.00     0.00  0.00    0.41  0.00  0.00   0.0   0.00  0.00  ...   

   respond   sad   san  shameless  smile  sons  sooo  soooo  would   wow  
0     0.58  0.00  0.00       0.00   0.00  0.00  0.00   0.00   0.58  0.00  
1 

In [219]:
import gensim
import nltk

#Tokenizing corpora.
TOKENIZED_CORPUS = [nltk.word_tokenize(sentence) for sentence in CORPUS]
tokenized_new_doc = [nltk.word_tokenize(sentence) for sentence in new_doc]
#TOKENIZED_CORPUS = CORPUS
#tokenized_new_doc = new_doc

print(TOKENIZED_CORPUS)
#Building the word2vec model on our training corpus
model = gensim.models.Word2Vec(TOKENIZED_CORPUS, size = 10, window = 10, min_count = 1, sample = 1e-3)

#Printing the vector for an example.
print(model['would'])

[['i', 'would', 'respond', 'i', 'go'], ['sooo', 'sad', 'i', 'miss', 'san', 'diego'], ['boss', 'bully'], ['interview', 'leave', 'alone'], ['sons', 'could', 'put', 'release', 'already', 'buy'], ['shameless', 'plug', 'best', 'rangers', 'forum', 'earth'], ['2am', 'feed', 'baby', 'fun', 'smile', 'coo'], ['soooo', 'high'], ['both'], ['journey', 'wow', 'u', 'become', 'cooler', 'hehe', 'possible']]
[-0.0120806  -0.04740473  0.00549438  0.02106401 -0.01556838  0.04457872
 -0.01424363 -0.00197872 -0.04770748 -0.01278542]


C:\Users\Bhair\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [220]:
# define function to compute tfidf weighted averaged word vector for a document
def tfidf_wtd_avg_word_vectors(words, tfidf_vector, tfidf_vocabulary, model, num_features):
    word_tfidfs = [tfidf_vector[0, tfidf_vocabulary.get(word)] 
                   if tfidf_vocabulary.get(word) 
                   else 0 
                   for word in words]
    
    word_tfidf_map = {word:tfidf_val 
                      for word, tfidf_val in zip(words, word_tfidfs)}
    feature_vector = np.zeros((num_features,), dtype = "float64")
    vocabulary = set(model.wv.index2word)
    wts = 0.
    for word in words:
        if word in vocabulary:
            word_vector = model[word]
            weighted_word_vector = word_tfidf_map[word] * word_vector
            wts = wts + word_tfidf_map[word]
            feature_vector = np.add(feature_vector, weighted_word_vector)
    if wts:
        feature_vector = np.divide(feature_vector, wts)
    return feature_vector


# generalize above function for a corpus of documents
def tfidf_weighted_averaged_word_vectorizer(corpus, tfidf_vectors, tfidf_vocabulary, model, num_features):
    docs_tfidfs = [(doc, doc_tfidf) 
                   for doc, doc_tfidf in zip(corpus, tfidf_vectors)]
    
    features = [tfidf_wtd_avg_word_vectors(tokenized_sentence, tfidf, tfidf_vocabulary, model, num_features)
                for tokenized_sentence, tfidf in docs_tfidfs]
    return np.array(features)


In [221]:
# get tfidf weights and vocabulary from earlier results and compute result
corpus_tfidf = tdidf_features
vocab = tfidf_vectorizer.vocabulary_
wt_tfidf_word_vec_features = tfidf_weighted_averaged_word_vectorizer(corpus = TOKENIZED_CORPUS, 
                                                                     tfidf_vectors = corpus_tfidf,
tfidf_vocabulary = vocab, model = model, num_features = 10)
print(np.round(wt_tfidf_word_vec_features, 3))

# compute avgd word vector for test new_doc
nd_wt_tfidf_word_vec_features = tfidf_weighted_averaged_word_vectorizer(corpus = tokenized_new_doc, 
                                                                        tfidf_vectors = nd_tfidf, 
                                                                        tfidf_vocabulary = vocab, model = model, 
                                                                        num_features = 10)
print(np.round(nd_wt_tfidf_word_vec_features, 3))


[[-0.008 -0.023 -0.01  -0.008  0.002  0.006 -0.022  0.013 -0.029  0.013]
 [ 0.005 -0.014  0.003 -0.016  0.003 -0.015  0.007  0.009 -0.007  0.012]
 [ 0.009  0.015 -0.028  0.012 -0.007 -0.005 -0.014  0.043 -0.028  0.01 ]
 [ 0.013  0.013  0.022 -0.014  0.023 -0.018  0.017  0.003  0.006  0.015]
 [-0.005 -0.     0.018 -0.008 -0.012  0.009  0.015  0.014  0.011  0.019]
 [ 0.006 -0.019  0.006 -0.007 -0.    -0.006 -0.007  0.007 -0.007 -0.008]
 [-0.023  0.007  0.023 -0.    -0.004  0.007 -0.01   0.029  0.011  0.011]
 [-0.006  0.002 -0.028 -0.002 -0.022 -0.004  0.027  0.026 -0.013  0.003]
 [ 0.015 -0.022 -0.014  0.021 -0.016  0.034 -0.041 -0.005  0.007 -0.038]
 [ 0.014  0.009 -0.003 -0.016  0.01  -0.    -0.017 -0.004 -0.003 -0.02 ]]
[[ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.   

C:\Users\Bhair\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [79]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [83]:

from gensim.test.utils import datapath
from gensim import utils

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('lee_background.cor')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)



In [84]:
import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences)

In [85]:
vec_king = model.wv['king']

In [87]:
for i, word in enumerate(model.wv.vocab):
    if i == 10:
        break
    print(word)

hundreds
of
people
have
been
forced
to
their
homes
in


In [89]:
new_model = gensim.models.Word2Vec(sentences = CORPUS)

RuntimeError: you must first build vocabulary before training the model

In [90]:
CORPUS

[['i', 'would', 'respond', 'i', 'go'],
 ['sooo', 'sad', 'i', 'miss', 'san', 'diego'],
 ['boss', 'bully'],
 ['interview', 'leave', 'alone'],
 ['sons', 'could', 'put', 'release', 'already', 'buy'],
 ['shameless', 'plug', 'best', 'rangers', 'forum', 'earth'],
 ['2am', 'feed', 'baby', 'fun', 'smile', 'coo'],
 ['soooo', 'high'],
 ['both'],
 ['journey', 'wow', 'u', 'become', 'cooler', 'hehe', 'possible']]

In [99]:
try:
    vec_cameroon = model.wv['smile']
except KeyError:
    print("The word 'cameroon' does not appear in this model")

The word 'cameroon' does not appear in this model


In [103]:
from sklearn.feature_extraction.text import TfidfTransformer

In [206]:
import gensim
import nltk

#Tokenizing corpora.
TOKENIZED_CORPUS = [nltk.word_tokenize(sentence) for sentence in CORPUS]
tokenized_new_doc = [nltk.word_tokenize(sentence) for sentence in new_doc]
#TOKENIZED_CORPUS = CORPUS
#tokenized_new_doc = new_doc

print(TOKENIZED_CORPUS)
#Building the word2vec model on our training corpus
model = gensim.models.Word2Vec(TOKENIZED_CORPUS, size = 10, window = 10, min_count = 1, sample = 1e-3)

#Printing the vector for an example.
print(model['i'])

[['i', 'would', 'respond', 'i', 'go'], ['sooo', 'sad', 'i', 'miss', 'san', 'diego'], ['boss', 'bully'], ['interview', 'leave', 'alone'], ['sons', 'could', 'put', 'release', 'already', 'buy'], ['shameless', 'plug', 'best', 'rangers', 'forum', 'earth'], ['2am', 'feed', 'baby', 'fun', 'smile', 'coo'], ['soooo', 'high'], ['both'], ['journey', 'wow', 'u', 'become', 'cooler', 'hehe', 'possible']]
[ 0.04858509 -0.03346521 -0.03632755 -0.00503827 -0.02940086 -0.00320018
  0.02309685 -0.02500693 -0.0399977   0.03995411]


C:\Users\Bhair\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [82]:
model = keyedvectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)

In [6]:
data = data_normalization(train_data[:10])

In [7]:
data

['i would respond i go',
 'sooo sad i miss san diego',
 'boss bully',
 'interview leave alone',
 'sons could put release already buy',
 'shameless plug best rangers forum earth',
 '2am feed baby fun smile coo',
 'soooo high',
 'both',
 'journey wow u become cooler hehe possible']

In [12]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
def build_feature_matrix(documents, feature_type='frequency', ngram_range=(1, 1), min_df=0.0, max_df=1.0):
    feature_type = feature_type.lower().strip()
    if feature_type == 'binary':
        vectorizer = CountVectorizer(binary=True, min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    
    elif feature_type == 'frequency':
        vectorizer = CountVectorizer(binary=False, min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    
    elif feature_type == 'tfidf':
        vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    
    else:
        raise Exception("Wrong feature type entered. Possible values: 'binary', 'frequency', 'tfidf'")
    
    feature_matrix = vectorizer.fit_transform(documents).astype(float)
    return (vectorizer, feature_matrix)



In [15]:
build_feature_matrix(data, feature_type = 'tfidf', ngram_range = (1, 1), min_df = 0.0, max_df = 1.0)

(TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.float64'>, encoding='utf-8',
                 input='content', lowercase=True, max_df=1.0, max_features=None,
                 min_df=0.0, ngram_range=(1, 1), norm='l2', preprocessor=None,
                 smooth_idf=True, stop_words=None, strip_accents=None,
                 sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, use_idf=True, vocabulary=None),
 <10x40 sparse matrix of type '<class 'numpy.float64'>'
 	with 40 stored elements in Compressed Sparse Row format>)